In [1]:
import os
import cv2
import itertools
import librosa
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import soundfile as sf
import warnings
import zipfile
# from pydub import AudioSegment
from io import BytesIO
from tqdm import tqdm
from glob import glob
from PIL import Image
from google.colab import files
from google.cloud import storage
warnings.filterwarnings('ignore')

# Pytorch
import torch
import torchaudio
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from torchaudio.transforms import MelSpectrogram, MFCC, SpectralCentroid

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Pacotes para o relatório de hardware
import gc
import types
import pkg_resources
# import pytorch_lightning as pl

# Seed para reproduzir os mesmos resultados
np.random.seed(10)
torch.manual_seed(10)
torch.cuda.manual_seed(10)

In [2]:
device = 'cuda'

In [3]:
!find /content/sample_data/ -type f -exec rm {} +

### Import Data

In [4]:
firts_time = True

In [5]:
if firts_time:
    uploaded = files.upload()

    for fn in uploaded.keys():
        print('User uploaded file "{name}" with length {length} bytes'.format(
            name=fn, length=len(uploaded[fn])))


Saving projetos-aleatorios-379913-61df4a1c249e.json to projetos-aleatorios-379913-61df4a1c249e.json
User uploaded file "projetos-aleatorios-379913-61df4a1c249e.json" with length 2372 bytes


In [6]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = f'/content/projetos-aleatorios-379913-61df4a1c249e.json'

In [7]:
storage_client = storage.Client()

In [8]:
bucket_name = 'projeto_musical'
rar_file_name = 'dimensao_acordes.csv'
local_rar_path = '/content/' + rar_file_name

# Define o bucket e o blob
bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob(rar_file_name)

#Baixa o arquivo RAR para o ambiente local do Colab
blob.download_to_filename(local_rar_path)

In [9]:
bucket_name = 'projeto_musical'
rar_file_name = 'audio_data_var.zip'
local_rar_path = '/content/' + rar_file_name

# Define o bucket e o blob
bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob(rar_file_name)

#Baixa o arquivo RAR para o ambiente local do Colab
blob.download_to_filename(local_rar_path)

In [10]:
with zipfile.ZipFile('audio_data_var.zip', 'r') as zip_ref:
    # Extraindo todos os arquivos
    zip_ref.extractall('/content')

In [ ]:
dimensao_acordes = pd.read_csv('dimensao_acordes.csv')
dimensao_acordes.head()

In [ ]:
labels = pd.read_csv('audio_data.csv')
labels.drop(columns='Unnamed: 0',inplace=True)
labels.head()

### Converter em Imagem

In [ ]:
def audio_to_spectrogram_image(audio_path, image_save_path, image_transforms=None):
    y, sr = librosa.load(audio_path)

    start_sample = librosa.time_to_samples(0, sr=sr)
    end_sample = librosa.time_to_samples(1, sr=sr)

    # Cortando o áudio
    y = y[start_sample:end_sample]

    # Gera o espectrograma de Mel
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
    mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)

    # Salvando o espectrograma como imagem
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(mel_spectrogram_db, sr=sr, x_axis=None, y_axis=None)
    plt.axis('off')
    plt.savefig(image_save_path, bbox_inches='tight', pad_inches=0)
    plt.close()

    image.save(image_save_path)